In [1]:
# package for numeric computing
import numpy as np

# package for basic plotting
from matplotlib import pyplot as plt

# package for beamforming basics
import BeamformingBasics as bf


plt.close('all')

save_fig = False

In [2]:
# Create vector of time samples
fs = 48e3       # sampling freq [Hz]
dt = 1./fs      # temporal resolution [s]
T = 0.5           # length of recording [s]

N_time = int(T*fs)

# vector of time samples
t = np.linspace(0, T-dt, N_time)

In [3]:
# create uniform linear array

L = 1.4     # [m]
M = 15      

MyULA = bf.SensorArray(L, M)

Array inter sensor spacing is 0.09999999999999999 m


In [4]:
# define pulse characteristics: amplitude, center freq, duration, onset time
a0 = 1.
f0 = 5000            # [Hz]
T_pulse = 0.002      # [s]
N_pulse = int(T_pulse*fs)

x_pulse = bf.create_narrowband_pulse(a0, T_pulse, f0, fs)

# onset time
t_initial = 0.3

# speed of sound
c0 = 1500.

# acoustic wavelength of pulse center freq
ac_wavelength = c0/f0

print("Acoustic wavelength for f0 = {} Hz is {} m".format(f0, ac_wavelength))

if MyULA.d >= ac_wavelength/2:
    print('\tCareful --> Aliasing might occur!')

Acoustic wavelength for f0 = 5000 Hz is 0.3 m


In [5]:
# create array of sensor signals from a given direction of arrival
    
theta0_deg = 45.
p_array = bf.create_array_signals(MyULA, x_pulse, t_initial, T, theta0_deg, fs,
                                  c0, SNR_dB=60)

# plot some array signals
plt.figure()
for m in range(3):
    plt.subplot(3, 1, m+1)
    plt.plot(t, p_array[m + MyULA.M//2,:])
    plt.xlim([t_initial, t_initial+2*T_pulse])

plt.subplot(3, 1, 1)
plt.title('Some array signals')


__main__:14: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


Text(0.5, 1.0, 'Some array signals')

In [ ]:
# use delay-and-sum beamforming to map direction of arrival

N_theta = 181
theta = np.linspace(0, np.pi, N_theta)

weights = np.ones(M)

y_beamformer = bf.delayandsum_beamformer(MyULA, p_array, theta, weights, fs)

In [ ]:
# plot beamformer output for different steering directions
plt.figure()
plt.plot(t, y_beamformer[0, :], label='0 deg')
plt.plot(t, y_beamformer[45, :], '--', label='45 deg')
plt.plot(t, y_beamformer[90, :], ':', label='90 deg')
plt.xlim([t_initial, t_initial+2*T_pulse])
plt.legend()
plt.title('Beamformer output for different look directions')


# compare beamformer output to center sensor signal
plt.figure()
plt.plot(t, p_array[M//2, :], label='center sensor')
plt.plot(t, y_beamformer[45, :], ':', label='beamf output', color='C3')
plt.xlim([t_initial, t_initial+2*T_pulse])
plt.legend()
plt.title('Center sensor vs. beamformer output')


In [ ]:
# beamformer polar plot

y_beamf_polar = np.sum(y_beamformer**2, axis=1)

y_beamf_polar_dB = 10*np.log10(y_beamf_polar)
dB_max = y_beamf_polar_dB.max()
dyn_range = 25      # dB


fig = plt.figure()
ax = fig.add_subplot(111, polar=True)
ax.plot(theta, y_beamf_polar_dB)

# add arrow indicating direction of arrival of signal
ax.arrow(theta0_deg*np.pi/180, dB_max-dyn_range, 0, dyn_range, width=0.01,
         head_width=0.1, head_length=5,
         length_includes_head=True, edgecolor = 'red', facecolor = 'red')


ax.set_rmax(dB_max)
ax.set_rmin(dB_max - dyn_range)

ax.set_thetamin(0)
ax.set_thetamax(180)
fig.set_tight_layout(True)
